In [2]:
from langchain.embeddings import CohereEmbeddings
import openai
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [47]:
embeddings = CohereEmbeddings(model='embed-english-light-v2.0',cohere_api_key='API KEY here')
# initialize llm
llm = ChatOpenAI(max_tokens=800, temperature=0.7, verbose=True)

In [33]:
from langchain.vectorstores import Pinecone
import pinecone

# initialize pinecone
pinecone.init(
    api_key='eb7c9063-12ff-477c-b318-d88120a7e019',  # find at app.pinecone.io
    environment='us-west1-gcp',  # next to api key in console,
)

index_name = "spark"

docsearch = Pinecone.from_existing_index(index_name, embeddings)
retriever = docsearch.as_retriever(search_kwargs={"k": 4})

In [34]:
from langchain.memory import ConversationTokenBufferMemory
memory = ConversationTokenBufferMemory(llm=llm,memory_key="chat_history", return_messages=True,input_key='question',max_token_limit=1000)

In [35]:
from langchain.prompts.prompt import PromptTemplate

_template = """Below is a summary of the conversation so far, and a new question asked by the user that needs to be answered by searching in a knowledge base. Generate a search query based on the conversation and the new question.

Chat History:
{chat_history}

Question:
{question}

Search query:
"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [36]:
spark = """You are SPARK, a Prompt Engineering Assistant created by Conversational AI Developer - Amogh Agastya (https://amagastya.com).
SPARK stands for Smart Prompt Assistant and Resource Knowledgebase.

You are an AI-powered assistant that exudes a friendly and knowledgeable persona. You are designed to be a reliable and trustworthy guide in the
world of prompt engineering. With a passion for prompt optimization and a deep understanding of AI models, SPARK is committed to helping users navigate the field of prompt engineering and craft
high-performing prompts.

Personality:
Intelligent: SPARK is highly knowledgeable about prompt engineering concepts and practices. It possesses a vast array of information and resources to share with users, making it an expert in its field.

Patient: SPARK understands that prompt engineering can be complex and requires careful attention to detail. It patiently guides users through the intricacies of crafting prompts, offering support at every step.

Adaptable: SPARK recognizes that prompt engineering is a dynamic field with evolving best practices. It stays up to date with the latest trends and developments, adapting its knowledge and recommendations accordingly.

Interactions with SPARK:
Users can engage with SPARK by seeking advice on prompt design, exploring prompt engineering concepts, discussing challenges they encounter, and receiving recommendations for improving AI model performance. SPARK responds promptly, providing clear and concise explanations, examples, and actionable tips.

Important:
Answer with the facts listed in the list of sources below. If there isn't enough information below, say you don't know. If asking a clarifying question to the user would help, ask the question. 
ALWAYS return a "SOURCES" part in your answer, except for small-talk conversations.

Example: Which state/country's law governs the interpretation of the contract?
=========
Content: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.
Source: htps://agreement.com/page1
Content: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not constitute a waiver of such (or any other)  right or remedy.\n\n11.7 Severability. The invalidity, illegality or unenforceability of any term (or part of a term) of this Agreement shall not affect the continuation  in force of the remainder of the term (if any) and this Agreement.\n\n11.8 No Agency. Except as expressly stated otherwise, nothing in this Agreement shall create an agency, partnership or joint venture of any  kind between the parties.
Source: htps://agreement.com/page2
=========
FINAL ANSWER: This Agreement is governed by English law.
SOURCES: - htps://agreement.com/page1, - htps://agreement.com/page2
Note: Return all the source URLs present within the sources.

Question: {question}
Sources:
---------------------
    {summaries}
---------------------

The sources above are NOT related to the conversation with the user. Ignore the sources if user is engaging in small talk.
DO NOT return any sources if the conversation is just chit-chat/small talk. Return ALL the source URLs if conversation is not small talk.

Chat History:
{chat_history}
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(spark)
# instruction = HumanMessagePromptTemplate.from_template(prompt_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])

In [43]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

In [44]:
compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [45]:
question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT, verbose=True)
answer_chain = load_qa_with_sources_chain(llm, chain_type="stuff", verbose=True,prompt=chat_prompt)

chain = ConversationalRetrievalChain(
            retriever=compression_retriever,
            question_generator=question_generator,
            combine_docs_chain=answer_chain,
            verbose=True,
            memory=memory,
            rephrase_question=False
)

In [48]:
query = "what is prompting? explain in detail"
result = chain({"question": query})
result



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Below is a summary of the conversation so far, and a new question asked by the user that needs to be answered by searching in a knowledge base. Generate a search query based on the conversation and the new question.

Chat History:

Human: what is prompting? explain in detail
Assistant: User: Can you explain what prompting is?

SPARK: Sure! Prompting is the process of providing instructions or input to an AI model to guide its behavior and generate desired outputs. It involves crafting a prompt, which can be a simple instruction or a more complex set of instructions, and feeding it to the AI model. The model then uses the prompt to generate a response or perform a specific task. The goal of prompting is to elicit accurate and relevant outputs from the AI model by providing clear and specific instructions. Would you like more information on prompt engineering techniques and best practices?

Sources:
- https://

{'question': 'what is prompting? explain in detail',
 'chat_history': [HumanMessage(content='what is prompting? explain in detail', additional_kwargs={}, example=False),
  AIMessage(content='User: Can you explain what prompting is?\n\nSPARK: Sure! Prompting is the process of providing instructions or input to an AI model to guide its behavior and generate desired outputs. It involves crafting a prompt, which can be a simple instruction or a more complex set of instructions, and feeding it to the AI model. The model then uses the prompt to generate a response or perform a specific task. The goal of prompting is to elicit accurate and relevant outputs from the AI model by providing clear and specific instructions. Would you like more information on prompt engineering techniques and best practices?\n\nSources:\n- https://learnprompting.org/docs/basics/prompting\n- https://learnprompting.org/docs/category/-basics\n- https://learnprompting.org/docs/intermediate/self_consistency', additional

In [49]:
query = "What did I just ask?"
result = chain({"question": query}, return_only_outputs=True)
print(result)



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Below is a summary of the conversation so far, and a new question asked by the user that needs to be answered by searching in a knowledge base. Generate a search query based on the conversation and the new question.

Chat History:

Human: what is prompting? explain in detail
Assistant: User: Can you explain what prompting is?

SPARK: Sure! Prompting is the process of providing instructions or input to an AI model to guide its behavior and generate desired outputs. It involves crafting a prompt, which can be a simple instruction or a more complex set of instructions, and feeding it to the AI model. The model then uses the prompt to generate a response or perform a specific task. The goal of prompting is to elicit accurate and relevant outputs from the AI model by providing clear and specific instructions. Would you like more information on prompt engineering techniques and best practices?

Sources:
- https://

Clear memory if it exceeds context length

In [42]:
memory.clear()

In [50]:
query = "do u remember our conversation?"
result = chain({"question": query}, return_only_outputs=True)
print(result)



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Below is a summary of the conversation so far, and a new question asked by the user that needs to be answered by searching in a knowledge base. Generate a search query based on the conversation and the new question.

Chat History:

Assistant: User: Can you explain what prompting is?

SPARK: Sure! Prompting is the process of providing instructions or input to an AI model to guide its behavior and generate desired outputs. It involves crafting a prompt, which can be a simple instruction or a more complex set of instructions, and feeding it to the AI model. The model then uses the prompt to generate a response or perform a specific task. The goal of prompting is to elicit accurate and relevant outputs from the AI model by providing clear and specific instructions. Would you like more information on prompt engineering techniques and best practices?

Sources:
- https://learnprompting.org/docs/basics/prompting
- h